<a href="https://colab.research.google.com/github/Anshi13713/ams/blob/main/Advanced_AI_Driven_Attendance_Management_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install opencv-python-headless
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import os
from datetime import datetime
from google.colab.patches import cv2_imshow

# JavaScript functions to access webcam and convert images
def js_to_image(js_reply):
    """
    Params:
            js_reply: JavaScript object containing image from webcam
    Returns:
            img: OpenCV BGR image
    """
    # decode base64 image
    image_bytes = b64decode(js_reply.split(',')[1])
    # convert bytes to numpy array
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    # decode numpy array into OpenCV BGR image
    img = cv2.imdecode(jpg_as_np, flags=1)

    return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
    """
    Params:
            bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
    Returns:
          bytes: Base64 image byte string
    """
    # convert array into PIL image
    bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
    iobuf = io.BytesIO()
    # format bbox into png for return
    bbox_PIL.save(iobuf, format='png')
    # format return string
    bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

    return bbox_bytes

# Load images and encode them
def load_images(path):
    images = []
    Names = []
    myList = os.listdir(path)
    print("Loading images...")
    for cl in myList:
        curImg = cv2.imread(os.path.join(path, cl))
        images.append(curImg)
        Names.append(os.path.splitext(cl)[0])
    print("Images loaded successfully.")
    return images, Names

def find_encodings(images):
    encodeList = []
    print("Encoding images...")
    for im in images:
        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        encode = encode_image(im)
        encodeList.append(encode)
    print("Encoding completed.")
    return encodeList

def encode_image(image):
    # Implement your encoding logic here
    pass

marked_present = set()  # To keep track of names already marked present

def mark_attendance(name):
    global marked_present
    if name not in marked_present:
        with open('Attendance.csv', 'a+') as f:
            now = datetime.now()
            dt_string = now.strftime('%I:%M:%S')
            f.write(f'{name},{dt_string}\n')
            print(f"{name} marked present at {dt_string}")
        marked_present.add(name)

def detect_faces(image):
    # Implement your own face detection method here
    pass

def main():
    path = 'train_images'
    images, Names = load_images(path)
    encodeListKnow = find_encodings(images)

    while True:
        js = Javascript('''
            async function takePhoto(quality) {
              const div = document.createElement('div');
              const capture = document.createElement('button');
              capture.textContent = 'Capture';
              div.appendChild(capture);

              const video = document.createElement('video');
              video.style.display = 'block';
              const stream = await navigator.mediaDevices.getUserMedia({video: true});

              document.body.appendChild(div);
              div.appendChild(video);
              video.srcObject = stream;
              await video.play();

              // Resize the output to fit the video element.
              google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

              // Wait for Capture to be clicked.
              await new Promise((resolve) => capture.onclick = resolve);

              const canvas = document.createElement('canvas');
              canvas.width = video.videoWidth;
              canvas.height = video.videoHeight;
              canvas.getContext('2d').drawImage(video, 0, 0);
              stream.getVideoTracks()[0].stop();
              div.remove();
              return canvas.toDataURL('image/jpeg', quality);
            }
            ''')
        display(js)

        # get photo data
        try:
            data = eval_js('takePhoto({})'.format(0.8))
            # get OpenCV format image
            img = js_to_image(data)
            ims = cv2.resize(img, (0, 0), None, 0.25, 0.25)
        except Exception as e:
            print(f"Error: {e}")
            break

        facesCurFrame = detect_faces(ims)
        encodesCurFrame = find_encodings(facesCurFrame)

        for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
            matches = compare_faces(encodeListKnow, encodeFace)
            faceDis = calculate_distance(encodeListKnow, encodeFace)
            matchIndex = np.argmin(faceDis)

            if matches[matchIndex]:
                name = Names[matchIndex].upper()
                y1, x2, y2, x1 = faceLoc
                y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
                cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
                mark_attendance(name)

        cv2.imshow('camera', img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break

if __name__ == "__main__":
    main()


Loading images...
Images loaded successfully.
Encoding images...
Encoding completed.


<IPython.core.display.Javascript object>

Encoding images...


TypeError: 'NoneType' object is not iterable

In [44]:
from IPython.display import display, Javascript, Image, HTML
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import os

# Function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
    """
    Params:
            js_reply: JavaScript object containing image from webcam
    Returns:
            img: OpenCV BGR image
    """
    # Decode base64 image
    image_bytes = b64decode(js_reply.split(',')[1])
    # Convert bytes to numpy array
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    # Decode numpy array into OpenCV BGR image
    img = cv2.imdecode(jpg_as_np, flags=1)

    return img

# Function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
    """
    Params:
            bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
    Returns:
          bytes: Base64 image byte string
    """
    # Convert array into PIL image
    bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
    iobuf = io.BytesIO()
    # Format bbox into png for return
    bbox_PIL.save(iobuf, format='png')
    # Format return string
    bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

    return bbox_bytes

# Initialize the Haar Cascade face detection model
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

# Function to load known faces and their encodings
def load_known_faces():
    known_faces_encodings = {}
    for file in os.listdir('train_images'):
        name = os.path.splitext(file)[0]
        face_img = cv2.imread(os.path.join('train_images', file))
        face_encoding = encode_face(face_img)
        known_faces_encodings[name] = face_encoding
    return known_faces_encodings

# Function to encode face
def encode_face(face_img):
    face_encoding = None
    gray = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        face_encoding = gray[y:y+h, x:x+w]
    return face_encoding

# Function to mark attendance
def mark_attendance(name):
    with open('attendance.txt', 'a') as f:
        f.write(f'{name}, {time.strftime("%Y-%m-%d %H:%M:%S")}\n')

# Function to take photo when face detected
def take_photo(filename='photo.jpg', quality=0.8):
    # Get photo data
    data = eval_js('takePhoto({})'.format(quality))
    # Get OpenCV format image
    img = js_to_image(data)
    # Grayscale image
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    print(gray.shape)
    # Get face bounding box coordinates using Haar Cascade
    faces = face_cascade.detectMultiScale(gray)
    # Draw face bounding box on image
    for (x,y,w,h) in faces:
        img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    # Save image
    cv2.imwrite(filename, img)

    return filename

# HTML and JavaScript for UI/UX enhancement
HTML('''
    <script>
        async function takePhoto(quality) {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            capture.textContent = 'Capture';
            div.appendChild(capture);

            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});

            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();

            // Resize the output to fit the video element.
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

            // Wait for Capture to be clicked.
            await new Promise((resolve) => capture.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            div.remove();
            return canvas.toDataURL('image/jpeg', quality);
        }
    </script>
    <button onclick="takePhoto(0.8)">Take Photo</button>
''')

# Function to stream video from webcam
def video_stream():
    js = Javascript('''
        var video;
        var div = null;
        var stream;
        var captureCanvas;
        var imgElement;
        var labelElement;

        var pendingResolve = null;
        var shutdown = false;

        function removeDom() {
            stream.getVideoTracks()[0].stop();
            video.remove();
            div.remove();
            video = null;
            div = null;
            stream = null;
            imgElement = null;
            captureCanvas = null;
            labelElement = null;
        }

        function onAnimationFrame() {
            if (!shutdown) {
                window.requestAnimationFrame(onAnimationFrame);
            }
            if (pendingResolve) {
                var result = "";
                if (!shutdown) {
                    captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
                    result = captureCanvas.toDataURL('image/jpeg', 0.8)
                }
                var lp = pendingResolve;
                pendingResolve = null;
                lp(result);
            }
        }

        async function createDom() {
            if (div !== null) {
                return stream;
            }

            div = document.createElement('div');
            div.style.border = '2px solid black';
            div.style.padding = '3px';
            div.style.width = '100%';
            div.style.maxWidth = '600px';
            document.body.appendChild(div);

            const modelOut = document.createElement('div');
            modelOut.innerHTML = "<span>Status:</span>";
            labelElement = document.createElement('span');
            labelElement.innerText = 'No data';
            labelElement.style.fontWeight = 'bold';
            modelOut.appendChild(labelElement);
            div.appendChild(modelOut);

            video = document.createElement('video');
            video.style.display = 'block';
            video.width = div.clientWidth - 6;
            video.setAttribute('playsinline', '');
            video.onclick = () => { shutdown = true; };
            stream = await navigator.mediaDevices.getUserMedia(
                {video: { facingMode: "environment"}});
            div.appendChild(video);

            imgElement = document.createElement('img');
            imgElement.style.position = 'absolute';
            imgElement.style.zIndex = 1;
            imgElement.onclick = () => { shutdown = true; };
            div.appendChild(imgElement);

            const instruction = document.createElement('div');
            instruction.innerHTML =
                '<span style="color: red; font-weight: bold;">' +
                'When finished, click here or on the video to stop this demo</span>';
            div.appendChild(instruction);
            instruction.onclick = () => { shutdown = true; };

            video.srcObject = stream;
            await video.play();

            captureCanvas = document.createElement('canvas');
            captureCanvas.width = 640; //video.videoWidth;
            captureCanvas.height = 480; //video.videoHeight;
            window.requestAnimationFrame(onAnimationFrame);

            return stream;
        }

        async function stream_frame(label, imgData) {
            if (shutdown) {
                removeDom();
                shutdown = false;
                return '';
            }

            var preCreate = Date.now();
            stream = await createDom();

            var preShow = Date.now();
            if (label != "") {
                labelElement.innerHTML = label;
            }

            if (imgData != "") {
                var videoRect = video.getClientRects()[0];
                imgElement.style.top = videoRect.top + "px";
                imgElement.style.left = videoRect.left + "px";
                imgElement.style.width = videoRect.width + "px";
                imgElement.style.height = videoRect.height + "px";
                imgElement.src = imgData;
            }

            var preCapture = Date.now();
            var result = await new Promise(function(resolve, reject) {
                pendingResolve = resolve;
            });
            shutdown = false;

            return {'create': preShow - preCreate,
                    'show': preCapture - preShow,
                    'capture': Date.now() - preCapture,
                    'img': result};
        }
    ''')

    display(js)

# Function to display video frame
def video_frame(label, bbox):
    data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
    return data

# Start streaming video from webcam
video_stream()

# Label for video
label_html = 'Capturing...'
# Initialize bounding box to empty
bbox = ''
count = 0

# Load known faces and their encodings
known_faces_encodings = load_known_faces()

# Main loop for processing video frames
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # Convert JS response to OpenCV Image
    img = js_to_image(js_reply["img"])

    # Create transparent overlay for bounding box
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    # Grayscale image for face detection
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Get face region coordinates
    faces = face_cascade.detectMultiScale(gray)

    # Get face bounding box for overlay
    for (x,y,w,h) in faces:
        bbox_array = cv2.rectangle(bbox_array,(x,y),(x+w,y+h),(255,0,0),2)

        # Crop face region and encode
        face_region = gray[y:y+h, x:x+w]
        face_encoding = face_region

        # Compare face encoding with known faces
        for name, known_face_encoding in known_faces_encodings.items():
            # Compare face encodings
            if face_encoding is not None and known_face_encoding is not None:
                if cv2.face.distanceEuclidean(face_encoding, known_face_encoding) < 0.6:
                    # Mark attendance if face matches
                    mark_attendance(name)
                    print(f'{name} present')

    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # Convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    # Update bbox so next frame gets new overlay
    bbox = bbox_bytes


<IPython.core.display.Javascript object>